In [204]:
import pandas as pd
import datetime

# Zoom Data 

The Zoom data only stretches as far back as November 2020 so the data below is from November 2020 upto November 2, 2021

In [10]:
#File Paths Zoom data
nov_2020 = r'/Users/josorio/Downloads/meetings2020-11-01-2020-12-01.csv'
dec_2020 = r'/Users/josorio/Downloads/meetings2020-12-02-2021-01-01.csv'
jan_2021 = r'/Users/josorio/Downloads/meetings2021-01-02-2021-02-01.csv'
feb_2021 = r'/Users/josorio/Downloads/meetings2021-02-02-2021-03-02.csv'
mar_2021 = r'/Users/josorio/Downloads/meetings2021-03-03-2021-03-31.csv'
apr_2021 = r'/Users/josorio/Downloads/meetings2021-04-01-2021-04-29.csv'
may_2021 = r'/Users/josorio/Downloads/meetings2021-04-30-2021-05-29.csv'
jun_2021 = r'/Users/josorio/Downloads/meetings2021-05-30-2021-06-27.csv'
jul_2021 = r'/Users/josorio/Downloads/meetings2021-06-28-2021-07-26.csv'
aug_2021 = r'/Users/josorio/Downloads/meetings2021-07-27-2021-08-24.csv'
sep_2021 = r'/Users/josorio/Downloads/meetings2021-08-25-2021-09-22.csv'
oct_2021 = r'/Users/josorio/Downloads/meetings2021-09-23-2021-10-22.csv'
nov_2021 = r'/Users/josorio/Downloads/meetings2021-10-23-2021-11-21.csv'

In [11]:
zoom_data_2021 = [nov_2020, dec_2020, jan_2021, feb_2021, mar_2021, apr_2021, may_2021, jun_2021,
                 jul_2021, aug_2021, sep_2021, oct_2021, nov_2021]

In [26]:
#Combine Zoom monthly data

zoom_data_df = pd.DataFrame()

for data in zoom_data_2021:
    
    temp_df = pd.read_csv(data)
    
    zoom_data_df = zoom_data_df.append(temp_df)

zoom_data_df.reset_index(inplace = True)
    

We've had a total of 182,684 total meetings 

In [110]:
zoom_data_df.shape

(182684, 5)

Check what columns we have

In [28]:
zoom_data_df.columns

Index(['index', 'Meeting ID', 'Topic', 'Host', 'Email', 'User Type',
       'Department', 'Group', 'Start Time', 'End Time', 'Duration (hh:mm:ss)',
       'Participants', '3rd Party ID', 'Host Data Center', 'Phone', 'VoIP',
       '3rd Party Audio', 'Video', 'Screen Sharing', 'Recording', 'CRC',
       'Encryption', 'Cipher', 'Unnamed: 22'],
      dtype='object')

Let's only use the following columns and drop everything else

In [40]:
zoom_data_df = zoom_data_df.loc[:, ('Host', 'Email', 'Start Time', 'End Time', 'Duration (hh:mm:ss)')]

Sample dataframe

In [41]:
zoom_data_df.head()

,Host,Email,Start Time,End Time,Duration (hh:mm:ss)
0,Jason Hickey,jason@campaignmonitor.com,"Nov 01, 2020 10:16 AM",11:13 AM,00:56:07
1,Danielle Tovell,dtovell@campaignmonitor.com,"Nov 01, 2020 08:57 AM",09:18 AM,00:21:33
2,Sharon Strauss,sstrauss@campaignmonitor.com,"Nov 01, 2020 07:31 AM",08:00 AM,00:29:21
3,Jesal Shah,jesals@campaignmonitor.com,"Nov 01, 2020 05:55 AM",06:40 AM,00:44:35
4,Natalia Abal,nabal@liveclicker.com,"Nov 01, 2020 04:53 AM",07:32 AM,02:39:30


# Okta Directory

In [104]:
first = r'/Users/josorio/Downloads/first.json'
second = r'/Users/josorio/Downloads/second.json'
third = r'/Users/josorio/Downloads/third.json'
fourth = r'/Users/josorio/Downloads/fourth.json'
fifth = r'/Users/josorio/Downloads/fifth.json'
sixth = r'/Users/josorio/Downloads/sixth.json'

In [105]:
okta_data = [first, second, third, fourth, fifth, sixth]

In [107]:
#Combine Okta data

okta_df = pd.DataFrame()

for data in okta_data:
    
    temp_df = pd.read_json(data)
    okta_df = okta_df.append(temp_df)

okta_df.reset_index(inplace = True)

In [108]:
okta_df.shape

(1091, 13)

In [111]:
okta_df.columns

Index(['index', '_links', 'activated', 'created', 'credentials', 'id',
       'lastLogin', 'lastUpdated', 'passwordChanged', 'profile', 'status',
       'statusChanged', 'type'],
      dtype='object')

In [112]:
okta_df = okta_df.loc[:, ('profile', 'status')]

Make sure to only include ACTIVE users

In [149]:
okta_df = okta_df[okta_df.status == 'ACTIVE']

So far we have 1049 active users in Okta currently

In [163]:
okta_df.shape

(1049, 2)

Get the Emma and CM users based on what's on their costCenter attribute (either Emma, Inc or Campaign Monitor pty)

In [211]:
#Sample input

x = okta_df[0:1].profile
for i in x:
    print(i['email'])
    print(i['costCenter'])

rmurray@campaignmonitor.com
Emma, Inc.


In [159]:
emma_users = []
cm_users = []
error_users = []

for index, row in okta_df.iterrows():
    
    user_profile = row.profile
    user_email = user_profile['email']
    
    try:
        cost_center = user_profile['costCenter']

        if 'Emma' in cost_center:
            emma_users.append(user_email)

        elif 'Campaign' in cost_center:
            cm_users.append(user_email)
    except:
        error_users.append(user_email)
        

Number of Emma Users

In [160]:
len(emma_users)

218

Number of CM Users

In [161]:
len(cm_users)

138

# Determine the total meeting count and duration for CM and Emma

This is a helper function that converts string values into DateTime

In [192]:
def ReturnSeconds(duration):
    
    try:
        date_time = datetime.datetime.strptime(duration, "%H:%M:%S")
        a_timedelta = date_time - datetime.datetime(1900, 1, 1)
        seconds = a_timedelta.total_seconds()

        return seconds
    
    except:
        return 0

In [193]:
cm_meeting_count = 0
cm_meeting_duration = 0

emma_meeting_count = 0
emma_meeting_duration = 0

for index, row in zoom_data_df.iterrows():
    
    #check if the user is a CM user
    if row.Email in cm_users:
        
        cm_meeting_count += 1
        
        #get the meeting duration
        duration = row['Duration (hh:mm:ss)']
        seconds = ReturnSeconds(duration)
        
        cm_meeting_duration += seconds
    
    #check if the user is an Emma user
    if row.Email in emma_users:
        
        emma_meeting_count += 1
        
        #get the meeting duration
        duration = row['Duration (hh:mm:ss)']
        seconds = ReturnSeconds(duration)
        
        emma_meeting_duration += seconds

# Total CM Meeting Count

In [194]:
cm_meeting_count

20959

# Total Emma Meeting Count

In [195]:
emma_meeting_count

38999

In [196]:
#This is in seconds
cm_meeting_duration

50739170.0

In [203]:
#This is in seconds
emma_meeting_duration

80218809.0

# Total CM Meeting Duration

In [197]:
str(datetime.timedelta(seconds=cm_meeting_duration))

'587 days, 6:12:50'

# Total Emma Meeting Duration

In [198]:
str(datetime.timedelta(seconds=emma_meeting_duration))

'928 days, 11:00:09'